In [10]:
import random
import sys
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

600901/600901 [==============================] - 0s 1us/step


In [3]:
text = open(path).read().lower()
print(f'Corpus length: {len(text)}')

Corpus length: 600893


In [4]:
maxlen = 60
step = 3

sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


/tmp/ipykernel_4207/2588542983.py:16: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
/tmp/ipykernel_4207/2588542983.py:17: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


In [7]:
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128)               95232     
                                                                 
 dense_1 (Dense)             (None, 57)                7353      
                                                                 
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________


In [8]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

/home/rmaia/miniconda3/envs/dim0494/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [9]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [11]:
for epoch in range(1, 5):
    
    print(f'********* ****** EPOCH: {epoch} *****************\n')
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print(f'\n\n--- Generating with seed: {generated_text} ------\n\n')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print(f'\n\n------ temperature: {temperature} ------\n')
        sys.stdout.write(generated_text)
        
        for i in range(400):
            
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)

epoch 1


2022-11-03 11:02:19.850576: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


1565/1565 [==============================] - 13s 7ms/step - loss: 1.9711

--- Generating with seed: ationship, resulting in a mutual interest, a sort of allianc ------


------ temperature: 0.2 ------

ationship, resulting in a mutual interest, a sort of alliance of the science of the the propection and string the man in the such a selfestic and the present in the such a man in the such a such a man intere concestions and destreation of the such a some intere all the such the man in the concession of the such a man, and string--the propection and the mankind in the such a man in the such a man in the are came and the precises the intere stinct of the pro
------ temperature: 0.5 ------

n the are came and the precises the intere stinct of the propection with thought is the sum the not in the extress of aster to the intentence with the the prepistion as which it is its light
of the not the him the presires as a menders in the heart the exiren a man in the profuct its the strences of phil

d over oney, here for vaye all dose of relations of has screading always, with the part may a comes mo eurom tower peoplereate," and stitiouman shotaks becauses so thrine by its vilt, awandly, or go ruriound, attestem, embleaveringical is ith cater by grounde.c"--before thesele tmings" lewlary,
the
vigws on men and ffor the dispants" do not casedo-y and new
cluse
without mevent for those phisonomer, without
taste. heaght to the batious fare and amgain
seed